In [2]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import math

In [5]:
df = pd.read_csv('train_data.csv',index_col='Unnamed: 0')

In [6]:
df.head()

,t2m,NDVI,boundaryLayerHeight,rh,totalPrecipitation,windSpeed,windDirection,SWIR1,NIR,red,...,day_sin,season_cos,season_sin,week_cos,week_sin,month_cos,month_sin,Grasses_avg_daily,Grasses_std_daily,Grasses
0,-0.369523,0.4982,102.408676,88.53255,0.000732,0.649779,-135.56980,7903.0,8511.0,2964.0,...,0.017213,6.123234e-17,1.0,1.0,-2.449294e-16,0.866025,0.5,0.0,0.0,0.0
1,0.845408,0.4982,167.713550,94.78888,0.001186,1.490278,-153.75153,7482.0,6773.0,3226.0,...,0.017213,6.123234e-17,1.0,1.0,-2.449294e-16,0.866025,0.5,0.0,0.0,0.0
2,-3.842715,0.4982,62.996920,84.02357,0.000781,0.823700,162.49753,6292.0,7443.0,2565.0,...,0.017213,6.123234e-17,1.0,1.0,-2.449294e-16,0.866025,0.5,0.0,0.0,0.0
3,0.307328,0.4982,165.393590,96.01055,0.000915,1.141481,-117.63472,9590.0,10635.0,3852.0,...,0.017213,6.123234e-17,1.0,1.0,-2.449294e-16,0.866025,0.5,0.0,0.0,0.0
4,-11.214022,0.4982,27.412760,83.98948,0.000880,0.661277,156.37183,5678.0,8548.0,1672.0,...,0.017213,6.123234e-17,1.0,1.0,-2.449294e-16,0.866025,0.5,0.0,0.0,0.0


In [143]:
class MyDataset(Dataset):
    def __init__(self,dataset,transform=None):
        # data loading
        self.X = dataset.iloc[:,:-1].values.astype('float32')
        self.y = dataset.iloc[:,-1].values.reshape(-1,1).astype('float32')
        self.n_samples = dataset.shape[0]
        
        self.transform = transform

    def __len__(self):
        return self.n_samples
        
    def __getitem__(self,index):
        sample = self.X[index],self.y[index]
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
        

        

In [157]:
class ToTensor:
    def __call__(self,sample):
        inputs,targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)
    
class MulTransform:
    def __init__(self,factor):
        self.factor = factor
    def __call__(self,sample):
        inputs,targets = sample
        inputs *= self.factor
        return inputs, targets

In [159]:
composed = torchvision.transforms.Compose([ToTensor(),MulTransform(2)])

In [160]:
dataset = MyDataset(df,transform=composed)

In [161]:
feature, label = dataset[0]

In [162]:
feature.shape, label.shape

(torch.Size([38]), torch.Size([1]))

In [163]:
dataloader = DataLoader(dataset=dataset, batch_size=4096, shuffle=True, num_workers=1)

In [164]:
data = iter(dataloader).next()

In [165]:
feature, label = data

In [166]:
feature.shape, label.shape

(torch.Size([4096, 38]), torch.Size([4096, 1]))

In [167]:
n_epochs = 2
total_samples = len(dataset)
n_iters = math.ceil(total_samples/4096)
print(total_samples,n_iters)

102270 25


In [168]:
for epoch in range(n_epochs):
    for i, (inputs, labels) in enumerate(dataloader):
        # Forward, backward, update
        
        if (i+1) % 5 ==0:
            print(f'epoch {epoch+1}/{n_epochs}, step {i+1}/{n_iters}, inputs {inputs.shape}')

epoch 1/2, step 5/25, inputs torch.Size([4096, 38])
epoch 1/2, step 10/25, inputs torch.Size([4096, 38])
epoch 1/2, step 15/25, inputs torch.Size([4096, 38])
epoch 1/2, step 20/25, inputs torch.Size([4096, 38])
epoch 1/2, step 25/25, inputs torch.Size([3966, 38])
epoch 2/2, step 5/25, inputs torch.Size([4096, 38])
epoch 2/2, step 10/25, inputs torch.Size([4096, 38])
epoch 2/2, step 15/25, inputs torch.Size([4096, 38])
epoch 2/2, step 20/25, inputs torch.Size([4096, 38])
epoch 2/2, step 25/25, inputs torch.Size([3966, 38])
